wikipediaデータをダウンロード

In [2]:
%%time

import bz2
import shutil
from pathlib import Path

import requests


def download_file(url: str, output_path: Path) -> None:
    # ファイルのダウンロード
    print("Downloading...")
    response = requests.get(url, stream=True)
    with open(output_path, "wb") as file:
        shutil.copyfileobj(response.raw, file)
    print("Download complete.")


def decompress_bz2(input_path: Path, output_path: Path) -> None:
    # BZ2ファイルを解凍
    print("Decompressing...")
    with bz2.BZ2File(input_path, "rb") as input_file, open(
        output_path, "wb"
    ) as output_file:
        shutil.copyfileobj(input_file, output_file)
    print("Decompression complete.")


def download_and_decompress(url: str, output_dir: Path, output_filename: str) -> None:
    # 出力ディレクトリの確認と作成
    output_dir.mkdir(parents=True, exist_ok=True)
    print(f"Created directory {output_dir}")

    # ファイルのダウンロード
    temp_path = output_dir / (output_filename + ".bz2")
    download_file(url, temp_path)

    # BZ2ファイルを解凍
    decompressed_path = output_dir / output_filename
    decompress_bz2(temp_path, decompressed_path)

    # 一時ファイルを削除
    print("Cleaning up...")
    temp_path.unlink()
    print("Cleanup complete.")


url = (
    "https://dumps.wikimedia.org/jawiki/20240401/jawiki-20240401-pages-articles.xml.bz2"
)
output_dir = Path("./data")
output_filename = "jawiki-20240401-pages-articles.xml"

download_and_decompress(url, output_dir, output_filename)


CPU times: user 32.6 ms, sys: 1.66 ms, total: 34.3 ms
Wall time: 33.7 ms


## テキストデータの抽出し、データクレンジング  
テキストクレンジングではneologdnを利用して、全角・半角の統一などを行なっている

In [16]:
%%time
import unicodedata

import neologdn

# xmlからテキストを抽出
!python -m wikiextractor.WikiExtractor ./data/jawiki-20240401-pages-articles.xml -b 10G --json

# ファイルをjsonl形式に変更
wiki_path = Path("./text/AA/wiki_00")
rename_path = Path("./text/AA/wiki_00.jsonl")
wiki_path.rename(rename_path)
wiki_path = rename_path

# 文字コードの変換
output_file = output_dir/"wiki.jsonl"

# 出力ファイルを開く
with open(output_file, "w", encoding="utf-8") as output_file:
    # 入力ファイルを開いて、一行ずつ処理
    with open(wiki_path, "r", encoding="ascii") as input_file:
        for line in input_file:
            # 各行をJSONオブジェクトとして解析
            data = json.loads(line)

            # データの正規化
            normalized_text = neologdn.normalize(data["text"])
            normalized_text = unicodedata.normalize("NFKC", normalized_text)
            data["normalized_text"] = normalized_text
            
            # JSONオブジェクトをJSON Lines形式で書き出し
            output_file.write(json.dumps(data, ensure_ascii=False) + "\n")

shutil.rmtree("./text")

CPU times: user 7min 51s, sys: 7.9 s, total: 7min 59s
Wall time: 15min 30s


テキストデータの抽出

In [3]:
%%time

import mwxml
import json

dump_file = output_dir/output_filename

with open(output_dir/"wiki.jsonl", 'a', encoding='utf-8') as output_file:
    for page in mwxml.Dump.from_file(dump_file):
        # 通常の記事であり、リダイレクトページでないものを抽出
        if (page.namespace == 0) and (page.redirect is None):
            id = page.id
            title = page.title
            for revision in page:
                text = revision.text
                if text:
                    break
    
            # 記事のタイトルとテキストをJSON形式に変換する
            article_json = json.dumps({
                'id': id,
                'title': title,
                'text': text,
            }, ensure_ascii=False)
    
            # JSONをファイルに書き込む
            output_file.write(article_json + '\n')

CPU times: user 8min 12s, sys: 14.6 s, total: 8min 27s
Wall time: 8min 28s


In [4]:
output_file

<_io.TextIOWrapper name='data/wiki.jsonl' mode='a' encoding='utf-8'>

In [20]:
dump = mwxml.Dump.from_file(dump_file)

In [30]:
for revision in page:
    text = revision.text
    if text:
        break

In [31]:
text

'{{Redirect|&}}\n{{Otheruses|記号|競走馬|アンパサンド (競走馬)}}\n{{WikipediaPage|「アンパサンド (＆)」の使用|WP:JPE#具体例による説明}}\n{{複数の問題|出典の明記=2018年10月8日 (月) 14:50 (UTC)|独自研究=2018年10月8日 (月) 14:50 (UTC)}}\n{{記号文字|&amp;}}\n[[File:Trebuchet MS ampersand.svg|thumb|100px|[[Trebuchet MS]] フォント]]\n\'\'\'アンパサンド\'\'\'（\'\'\'&amp;\'\'\', {{Lang-en|ampersand}}）は、並立助詞「…と…」を意味する[[記号]]である。[[ラテン語]]で「…と…」を表す接続詞 "et" の[[合字]]を起源とする。現代のフォントでも、[[Trebuchet MS]] など一部のフォントでは、"et" の合字であることが容易にわかる字形を使用している。\n\n== 語源 ==\n{{quote|The term ampersand is a corruption of and (&) per se and, which literally means "(the character) & by itself (is the word) and." The symbol & is derived from the ligature of ET or et, which is the Latin word for "and."<br>\n訳: アンパサンドという言葉は、\'\'and (&) per se and\'\'（"&" という文字それ自体が "and" という言葉を意味する）が[[言語変化|転訛]]したものである。& の記号は、ラテン語で "and" を意味する "[[:wikt:et|ET]]" または et の[[合字]]が元になっている。|source = Geoffrey Glaister, \'\'Glossary of the Book\'\'<ref name="adobe">{{cite book |last=[[:en:Geoffrey Glaister|Glaister]]|firs

In [27]:
page

Page(id=22215, title='セッケン', namespace=0, redirect='石鹸', restrictions=[])